# Fix Seed

In [1]:
import random
import torch
import numpy as np

# 시드 고정 함수 정의
def seed_everything(seed=5):
    # Python의 기본 random 모듈 시드 설정
    random.seed(seed)
    
    # NumPy의 난수 생성 시드 설정
    np.random.seed(seed)
    
    # PyTorch의 CPU 난수 생성 시드 설정
    torch.manual_seed(seed)
    
    # CUDA가 사용 가능한 경우 GPU 난수 생성 시드 설정
    if torch.cuda.is_available():
        # 현재 GPU 장치에 대한 시드 설정
        torch.cuda.manual_seed(seed)
        # 모든 GPU 장치에 대한 시드 설정
        torch.cuda.manual_seed_all(seed)

    # PyTorch의 CuDNN 설정을 통해 재현성을 보장
    # CuDNN이 제공하는 최적화가 정확하게 재현될 수 있도록 보장
    torch.backends.cudnn.deterministic = True  # 고정된 알고리즘만 사용하도록 설정
    torch.backends.cudnn.benchmark = False  # 입력 크기가 고정된 경우 최적화 비활성화

seed_everything()

# Dataset

In [2]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, dir_paths, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform

        for dir_path in dir_paths:
            label = os.path.basename(dir_path) # 'cat' 또는 'dog'
            for file_name in os.listdir(dir_path):
                image_path = os.path.join(dir_path, file_name)
                image = Image.open(image_path).convert('RGB')
                if self.transform:
                    image = self.transform(image)
                self.images.append(image)
                self.labels.append(label)
                
        # 레이블 인코딩
        self.label_to_index = {'cat': 0, 'dog': 1}
        self.labels = [self.label_to_index[label] for label in self.labels]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = image.float() / 255.0  # Normalize to [0, 1]

        return image, label


In [3]:
from torchvision import transforms

# 데이터 전처리 정의 (Data Augmentation 추가)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(45),
    transforms.RandomResizedCrop(32, scale=(0.5, 1.0)),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.3),
    transforms.RandomGrayscale(p=0.3),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.ToTensor(),
])

# 디렉터리 경로
cat_dir = '../data/cifar10_images/cat'
dog_dir = '../data/cifar10_images/dog'

# Custom Dataset 인스턴스 생성
dataset = CustomDataset(dir_paths=[cat_dir, dog_dir], transform=transform)

# 데이터셋을 학습용, 검증용, 테스트용으로 분리
train_size = int(0.7 * len(dataset))
valid_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

# DataLoader 인스턴스 생성
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 데이터 확인
for images, labels in train_loader:
    print(f"학습 배치 이미지 크기: {images.size()}")
    print(f"학습 배치 레이블 크기: {labels.size()}")
    break

for images, labels in valid_loader:
    print(f"검증 배치 이미지 크기: {images.size()}")
    print(f"검증 배치 레이블 크기: {labels.size()}")
    break

for images, labels in test_loader:
    print(f"테스트 배치 이미지 크기: {images.size()}")
    print(f"테스트 배치 레이블 크기: {labels.size()}")
    break

학습 배치 이미지 크기: torch.Size([16, 3, 32, 32])
학습 배치 레이블 크기: torch.Size([16])
검증 배치 이미지 크기: torch.Size([16, 3, 32, 32])
검증 배치 레이블 크기: torch.Size([16])
테스트 배치 이미지 크기: torch.Size([16, 3, 32, 32])
테스트 배치 레이블 크기: torch.Size([16])


# HyperParameter

In [4]:
# 하이퍼파라미터 설정
learning_rate = 0.0001
num_epochs = 100
weight_decay = 1e-5  # L2 정규화 하이퍼파라미터
early_stopping_patience = 100
early_stopping_counter = 20

# Weight Initalize

In [5]:
import torch.nn as nn

# 가중치 초기화 함수
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# Model

In [6]:
import torch.nn.functional as F


class EnhancedCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(EnhancedCNN, self).__init__()
        
        # Convolutional Block 1
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Convolutional Block 2
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        # Convolutional Block 3
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(512 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(p=0.5)
        
        # Initialize weights
        self.apply(weights_init)

    def forward(self, x):
        # Convolutional Block 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        
        # Convolutional Block 2
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        
        # Convolutional Block 3
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return torch.sigmoid(x)

model = EnhancedCNN(input_channels=3, num_classes=1)

In [7]:
def count_parameters(model):
    """
    모델의 파라미터 수를 계산하는 함수
    """
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

def print_model_parameters(model):
    """
    모델의 각 레이어와 그 파라미터 수를 출력하는 함수
    """
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel()} parameters")
    print(f"Total parameters: {count_parameters(model)}")

# 모델의 파라미터 수 출력
print_model_parameters(model)

conv1.weight: 864 parameters
conv1.bias: 32 parameters
bn1.weight: 32 parameters
bn1.bias: 32 parameters
conv2.weight: 18432 parameters
conv2.bias: 64 parameters
bn2.weight: 64 parameters
bn2.bias: 64 parameters
conv3.weight: 73728 parameters
conv3.bias: 128 parameters
bn3.weight: 128 parameters
bn3.bias: 128 parameters
conv4.weight: 294912 parameters
conv4.bias: 256 parameters
bn4.weight: 256 parameters
bn4.bias: 256 parameters
conv5.weight: 1179648 parameters
conv5.bias: 512 parameters
bn5.weight: 512 parameters
bn5.bias: 512 parameters
fc1.weight: 8388608 parameters
fc1.bias: 1024 parameters
fc2.weight: 524288 parameters
fc2.bias: 512 parameters
fc3.weight: 512 parameters
fc3.bias: 1 parameters
Total parameters: 10485505


# Loss

$
\begin{align}
-\frac{1}{N}\sum_{i=1}^N[y_ilog(p_i)+(1-y_i)log(1-p_i)]
\end{align}
$

1. 선형 모델의 출력: [0.5, -1.2, 0.8]
2. Sigmoid의 출력: [0.62245933, 0.23147522, 0.68997448]
3. 실제 레이블: [1, 0, 1]

In [8]:
# 이진 크로스 엔트로피 손실 함수
# 모델의 예측 확률과 실제 레이블 간의 차이를 측정하여, 예측이 정확할수록 손실을 줄이는 방향으로 모델을 학습시킵니다.
criterion = nn.BCELoss()  # Binary Cross Entropy Loss

# Optimizer

In [9]:
import torch.optim as optim

# Adam 옵티마이저 사용, L2 정규화 포함
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train

In [10]:
import matplotlib.pyplot as plt
import torch

# 조기 종료 설정
best_val_loss = float('inf')

# 학습 및 검증 손실을 저장할 리스트 초기화
train_losses = []
valid_losses = []

# 학습 루프 시작
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        labels = labels.float().view(-1, 1)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    model.eval()
    val_running_loss = 0.0

    with torch.no_grad():
        for images, labels in valid_loader:
            labels = labels.float().view(-1, 1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)

    val_epoch_loss = val_running_loss / len(valid_loader.dataset)
    valid_losses.append(val_epoch_loss)
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_epoch_loss:.4f}')

    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        early_stopping_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= early_stopping_patience:
        print("Early stopping")
        break

# 학습 및 검증 손실 시각화
epochs_range = range(1, len(train_losses) + 1)  # 실제 기록된 손실의 길이에 맞게 조정

plt.figure(figsize=(10, 5))
plt.plot(epochs_range, train_losses, label='Train Loss')
plt.plot(epochs_range, valid_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


Epoch 1/100, Loss: 1.5398
Epoch 1/100, Validation Loss: 0.6961
Epoch 2/100, Loss: 1.2770
Epoch 2/100, Validation Loss: 0.7287
Epoch 3/100, Loss: 1.1225
Epoch 3/100, Validation Loss: 0.7759
Epoch 4/100, Loss: 0.9381
Epoch 4/100, Validation Loss: 0.7566
Epoch 5/100, Loss: 0.9452
Epoch 5/100, Validation Loss: 0.7325
Epoch 6/100, Loss: 0.8602
Epoch 6/100, Validation Loss: 0.6732
Epoch 7/100, Loss: 0.9171
Epoch 7/100, Validation Loss: 0.6958
Epoch 8/100, Loss: 0.8084
Epoch 8/100, Validation Loss: 0.6838
Epoch 9/100, Loss: 0.8066
Epoch 9/100, Validation Loss: 0.6964
Epoch 10/100, Loss: 0.7707
Epoch 10/100, Validation Loss: 0.6931
Epoch 11/100, Loss: 0.7780
Epoch 11/100, Validation Loss: 0.6925
Epoch 12/100, Loss: 0.7677
Epoch 12/100, Validation Loss: 0.7027
Epoch 13/100, Loss: 0.7096
Epoch 13/100, Validation Loss: 0.7157
Epoch 14/100, Loss: 0.7407
Epoch 14/100, Validation Loss: 0.6980
Epoch 15/100, Loss: 0.6790
Epoch 15/100, Validation Loss: 0.6850
Epoch 16/100, Loss: 0.6586
Epoch 16/100, Va

KeyboardInterrupt: 

# Predict

In [ ]:
# 모델을 평가 모드로 설정
model.eval()
correct = 0
total = 0

# 평가 중 기울기 계산을 비활성화 (메모리 절약 및 연산 속도 향상)
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.float().view(-1, 1)
        outputs = model(images)
        predicted = (outputs > 0.5).float()

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 정확도 계산
accuracy = correct / total
# 정확도를 백분율로 출력
print(f"Test Set Accuracy: {accuracy * 100:.2f}%")